In [ ]:
# --- Step 1: Install Required Libraries ---
# pip install openai py2neo pandas

# --- Step 2: Connect to Neo4j ---
from py2neo import Graph, Node, Relationship
import ast

graph = Graph("neo4j://localhost:7687", auth=("neo4j", "testpassword"))

# --- Step 3: Define Graph Building Function ---
def build_graph_from_gpt_response(response_text):
    try:
        triples = ast.literal_eval(response_text)
    except Exception as e:
        print("Parsing Error:", e)
        return

    for triple in triples:
        if len(triple) != 3:
            continue
        subject, predicate, obj = [str(x).strip() for x in triple]
        subj_node = Node("Entity", name=subject)
        obj_node = Node("Entity", name=obj)
        rel = Relationship(subj_node, predicate, obj_node)
        graph.merge(subj_node, "Entity", "name")
        graph.merge(obj_node, "Entity", "name")
        graph.merge(rel)

# --- Step 4: Load Dataset and OpenAI Client ---
import pandas as pd
from openai import OpenAI

df = pd.read_csv("Procurement KPI Analysis Dataset.csv")
client = OpenAI(api_key="API_KEY_HERE")  # Replace with your OpenAI API key

# --- Step 5: Generate Triples and Build Graph ---
for i in range(5):  # use more rows later
    row = df.iloc[i]
    prompt = f"""
Extract subject-predicate-object triples from the following structured purchase order. Return the result as a valid Python list of 3-item tuples. Use proper literals — avoid vague terms like 'N/A'.

Example format:
[
  ("Delta Logistics", "supplies", "Raw Materials"),
  ("Raw Materials", "has_quantity", "1180"),
  ("Raw Materials", "has_price", "64.07"),
  ...
]

Purchase Order:
Supplier: {row['Supplier']}
Item Category: {row['Item_Category']}
Quantity: {row['Quantity']}
Unit Price: {row['Unit_Price']}
Order Status: {row['Order_Status']}
Compliance: {row['Compliance']}
"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    content = response.choices[0].message.content.strip()
    print(f"Triples for row {i}:", content)
    build_graph_from_gpt_response(content)

# --- Step 6: Querying the Knowledge Graph for RAG ---
# Example: Who supplies raw materials and is compliant?

question_query = """
MATCH (s:Entity)-[:supplies]->(c:Entity {name: "Raw Materials"})
MATCH (s)-[:Compliance]->(status:Entity {name: "Yes"})
RETURN s.name AS supplier
"""

suppliers = graph.run(question_query).data()
print("Compliant suppliers of Raw Materials:", suppliers)

# --- Step 7: Generate Natural Language Answer ---
def format_answer(suppliers):
    if not suppliers:
        return "No compliant suppliers found for Raw Materials."
    names = [s["supplier"] for s in suppliers]
    return f"The following compliant suppliers provide Raw Materials: {', '.join(names)}."

print(format_answer(suppliers))


Triples for row 0: [
  ("Alpha_Inc", "supplies", "Office Supplies"),
  ("Office Supplies", "has_quantity", "1176"),
  ("Office Supplies", "has_price", "20.13"),
  ("Office Supplies", "has_status", "Cancelled"),
  ("Office Supplies", "is_compliant", "Yes")
]
Triples for row 1: [
  ("Delta_Logistics", "supplies", "Office Supplies"),
  ("Office Supplies", "has_quantity", "1509"),
  ("Office Supplies", "has_price", "39.32"),
  ("Office Supplies", "order_status", "Delivered"),
  ("Office Supplies", "compliance", "Yes")
]
Triples for row 2: [
  ("Gamma_Co", "supplies", "MRO"),
  ("MRO", "has_quantity", "910"),
  ("MRO", "has_price", "95.51"),
  ("MRO", "order_status", "Delivered"),
  ("MRO", "compliance", "Yes")
]
Triples for row 3: [
  ("Beta_Supplies", "supplies", "Packaging"),
  ("Packaging", "has_quantity", "1344"),
  ("Packaging", "has_price", "99.85"),
  ("Packaging", "order_status", "Delivered"),
  ("Packaging", "compliance", "Yes")
]
Triples for row 4: [
  ("Delta_Logistics", "suppli

### Coverage

In [2]:
# Total number of unique suppliers in the CSV
total_suppliers = df['Supplier'].nunique()

# How many of those suppliers exist as named nodes in the graph?
query = """
MATCH (s:Entity)
WHERE EXISTS {
  MATCH (s)-[]->()
}
AND s.name IN $supplier_list
RETURN count(DISTINCT s.name) AS linked_suppliers
"""

linked_suppliers = graph.run(query, parameters={"supplier_list": df['Supplier'].dropna().unique().tolist()}).data()[0]['linked_suppliers']
coverage_percent = (linked_suppliers / total_suppliers) * 100

print(f"Supplier Node Coverage: {coverage_percent:.2f}%")

Supplier Node Coverage: 80.00%


### Accuracy

In [3]:
sample_check = graph.run("""  
MATCH (a:Entity)-[r]->(b:Entity)  
RETURN a.name AS subject, type(r) AS relation, b.name AS object  
LIMIT 10  
""").to_data_frame()  
  
print(sample_check)

         subject         relation           object
0       Supplier               is        Alpha_Inc
1       Supplier               is  Delta_Logistics
2       Supplier               is         Gamma_Co
3       Supplier        Alpha_Inc                 
4       Supplier  Delta_Logistics                 
5       Supplier         Gamma_Co                 
6  Item Category               is  Office Supplies
7  Item Category               is              MRO
8  Item Category  Office Supplies                 
9  Item Category              MRO                 


### Completeness

In [4]:
# Find suppliers without compliance links  
query = """  
MATCH (s:Entity)
WHERE s.name IN $supplier_list AND NOT (s)-[:Compliance]->()
RETURN s.name AS MissingCompliance
"""  

missing_compliance = graph.run(query, parameters={"supplier_list": df['Supplier'].dropna().unique().tolist()}).to_data_frame()  
print(missing_compliance)

  MissingCompliance
0         Alpha_Inc
1   Delta_Logistics
2          Gamma_Co
3     Beta_Supplies


### How Good Is the Knowledge Graph We Built?

In [5]:
# What % of suppliers in CSV have at least one edge in the graph?  
total_suppliers = df['Supplier'].nunique()

query = """
MATCH (s:Entity)
WHERE s.name IN $supplier_list
AND EXISTS {
  MATCH (s)-[]->()
}
RETURN count(DISTINCT s.name) AS linked_suppliers
"""

linked_suppliers = graph.run(query, parameters={"supplier_list": df['Supplier'].dropna().unique().tolist()}).data()[0]['linked_suppliers']
coverage_percent = (linked_suppliers / total_suppliers) * 100

print(f"Supplier Node Coverage: {coverage_percent:.2f}%")


Supplier Node Coverage: 80.00%
